# Network model

In this assignment, we will learn network models and how to use them by reproducing the results of the following paper:

> Colizza, V., Flammini, A., Serrano, M. et al. Detecting rich-club ordering in complex networks. Nature Phys 2, 110–115 (2006). https://doi.org/10.1038/nphys209

This paper explores the main factor that gives rise to a prevalent structure of complex networks, *rich-club* structure.

### What is rich club

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Disassortative_network_demonstrating_the_Rich_Club_effect.png/440px-Disassortative_network_demonstrating_the_Rich_Club_effect.png)

Rich club refers to a tendency for high-degree (rich) nodes to form a densely-connected group (club). The structure is first quantitatively identified in the structure of the Internet [[Zhou & Mondragon]](https://ieeexplore.ieee.org/document/1278314), which was later identified in social networks [[Guimerà, et al.]](https://www.science.org/doi/full/10.1126/science.1106340?casa_token=pBOtOBI3k_cAAAAA%3A64Amr-hwUeOFgO_Md23JLC8POrNwK4yFv6sDLjZua-yBjDsV108-qGexZ1P85KqjcWPVrinR3U2T1A).


### Quantitative evidence of rich club structure

The rich club structure is defined as follows.
Let us consider, for a moment, that nodes with a degree greater than $k$ are the members of the rich club.
Then, we measure the density of edges between the rich club members as **the rich-club coefficient**, denoted as $\psi(k)$, i.e.,
$$
\psi(k):=\frac{E_{>k}}{N_{>k}(N_{>k}-1)/2},
$$
where $N_{>k}$ is the number of nodes with degree greater than $k$ and $E_{>k}$ is the number of edges between them.
Essentially, $\psi(k)$ is the probability of edges within the rich club.
If the network has a rich club, the members of the rich club should be densely connected. Thus, $\psi(k)$ should increase if we increase the degree threshold $k$.

As an example, let us compute the rich club coefficient by using the Internet data taken from SNAP project: https://snap.stanford.edu/data/as-733.html

In [ ]:
from scipy import sparse
import pandas as pd
import igraph
import numpy as np
from scipy import stats


edge_table = pd.read_csv(
    "https://snap.stanford.edu/data/as20000102.txt.gz",
    skiprows=4,
    sep="\t",
    header=None,
    names=["src", "trg"],
)

# Reindexing
edge_table_flattened = edge_table[["src", "trg"]].values.reshape(
    -1
)  # flatten the edge table
reindexed_edge_table_flattened = np.unique(edge_table_flattened, return_inverse=True)[
    1
]  # Reinedxing
edge_table.loc[:, ["src", "trg"]] = reindexed_edge_table_flattened.reshape(
    (-1, 2)
)  # Resize

# Get edge ids
src, trg = tuple(edge_table[["src", "trg"]].values.T)

# Remove self loops
is_self_loop = src == trg
src = src[~is_self_loop]
trg = trg[~is_self_loop]

# Create igraph Graph object
edge_list = tuple(zip(src, trg))
g = igraph.Graph(edge_list, directed=False)
n_nodes = g.vcount()
n_edges = g.ecount()

# Create the adajacency matrix
A = g.get_adjacency_sparse()

print(f"\# of nodes = {n_nodes}, \# of edges = {n_edges}")

Let's compute the rich-club coefficient for degree $k$.

In [ ]:
def rich_club_coefficient(A, k):
    """Rich club coefficient

    Parameters
    ----------
    A : scipy CSR sparse martix
        Adjacency matrix
    k : int
        Degree threshold for the rich club

    Returns
    -------
    psi: float
        Rich club coefficient
    """
    psi ...
    return psi

Now, let's plot $\psi$ as a function of $k$:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

deg = np.array(A.sum(axis=1)).reshape(-1)

results = []
for k in np.unique(deg):
    psi = rich_club_coefficient(A, k)
    results.append({"k": k, "psi": psi})
plot_data = pd.DataFrame(results)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.lineplot(x="k", y="psi", data=plot_data, marker="o", ax=ax)
ax.set_xlabel("Degree, $k$")
ax.set_ylabel("Rich club coefficient")
ax.set_xscale("log")
ax.set_yscale("log")
sns.despine()

The figure illustrates a prominent rich club phenomenon in the Internet, as indicated by the significant upward trend in the rich club coefficient.

### Rich club in random networks

In the early 2000s, the rich club structure had been reported across networks in diverse domains. However, it remained unclear what gave rise to the seemingly universal rich club structure.

[Colliza et. al](https://www.nature.com/articles/nphys209) provided one plausible "cause" of the rich club phenomenon. Crucially, in the figure below, they demonstrated that even random networks (denoted by ER), which are supposed to have no structure at all, can have a similar increasing rich club coefficient.

This was a striking result since it suggests that the rich club---which had been considered to be a hallmark of complex networks---may be perhaps generated by some trivial mechanisms.

![](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fnphys209/MediaObjects/41567_2006_Article_BFnphys209_Fig1_HTML.jpg?as=webp)

## The origin of the rich-club phenomenon

Why does the random network have a strong, rich club even though the random network is supposed to have no non-trivial structures? It's because hub nodes have many edges and thus have a higher chance of being connected than nodes with few edges. Colizza et al. argue that the rich club is largely explained by node degree, meaning the rich club almost always exists if the degree has some variability and that the many empirical rich club structures can be explained by the degree.

Let us test the hypothesis by using the configuration model. The configuration model generates a random network with a given degree sequence. With `igraph`, you can create the configuration model by

In [ ]:
grand = g.copy()  # Make a copy
grand.rewire()  # This is in-place operation and break the data if you didn't make a copy

Now, let's compute the rich-club coefficient and compare it against the original network.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

deg = np.array(A.sum(axis=1)).reshape(-1)
Arand = grand.get_adjacency_sparse()

results = []
for k in np.unique(deg):
    psi = rich_club_coefficient(A, k)
    psi_rand = rich_club_coefficient(Arand, k)
    results.append({"k": k, "psi": psi, "psi_rand": psi_rand})
plot_data = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(5, 5))
sns.lineplot(x="k", y="psi", data=plot_data, marker="o", ax=ax, label="original")
sns.lineplot(
    x="k",
    y="psi_rand",
    data=plot_data,
    marker="o",
    ax=ax,
    label="Configuration model",
)
ax.set_xlabel("Degree, $k$")
ax.set_ylabel("Rich club coefficient")
ax.set_xscale("log")
ax.set_yscale("log")
sns.despine()

You should see that the rich club coefficient is very close to the one for the random networks, an indication that the rich club structure is largely due to the node degree.


## Normalized rich club coefficient

Colizza et al. develop a *normalized* rich club coefficient by reflecting the fact that hub nodes are likely to be connected to each other in random networks. The normalized rich club coefficient $\Phi_{\text{norm}}(k)$ is given by
$$
\Phi_{\text{norm}}(k) = \frac{\Phi(k)}{{\mathbb E}[\Phi_{\text{random}}(k)]}
$$
where $\Phi(k)$ is the original rich club coefficient for a given network, and ${\mathbb E}[\Phi_{\text{random}}(k)]$ is the original rich club coefficient for random networks generated by the configuration model, *on average*. We compute the average numerically by generating many realizations of the configuration models and take the average rich club coefficients of the realizations. The following is a sketch of how to compute the normalized rich club coefficient:

In [ ]:
def normalized_rich_club_coefficient(A, klist, n_random_graphs=30):
    """_summary_

    Parameters
    ----------
    A : scipy CSR sparse martix
        Adjacency matrix
    klist : numpy.ndarray
        list of degree thresholds
    n_random_graphs : int, optional
        Number of random networks, by default 30

    Returns
    -------
    rich_club_coefficients : list
        list of rich club coefficients
    """

    # Construct the igraph object
    src, trg, _ = sparse.find(sparse.triu(A, 1))
    edge_list = tuple(zip(src, trg))
    g = igraph.Graph(edge_list, directed=False)

    # Compute the rich club coefficient for the original graph
    rich_club_coefficients = []
    for k in klist:
        psi = rich_club_coefficient(A, k)
        rich_club_coefficients.append({"psi": psi, "k": k})

    # Generate a random graph and compute the rich club coefficient
    random_rich_club_coefficients = []
    for i in range(n_random_graphs):
        # Generate a random graph
        # Your code....

        for k in klist:
            # Compute the rich club coefficient for the random graph
            # Your code....

            # store the computed coefficient
            random_rich_club_coefficients.append({"psi_rand": psi_rand, "k": k})

    # Compute the average of psi_rand for each k
    # Your code....

    # Divided the rich club coefficient for the given network by the average of psi_rand with the same k
    # Your code....

    return normalized_psi_list

## Degree is a major determinant of network structure


Heterogeneous degree distribution can explain seemingly non-trivial network structure. An example is the centrality measure. Even if the centrality measures are not directly based on degree, such as the closeness centrality, they have a strong correlation with the degree.

Let us illustrate the strong correlation of several centrality measures to node degree. Let us compute the following centrality measures:


In [ ]:
centralities = []

# Page Rank
centralities += [{"centrality": g.pagerank(), "name": "pagerank"}]

# Closeness centrality
centralities += [{"centrality": g.closeness(), "name": "closeness"}]

# Betweenenss
centralities += [{"centrality": g.betweenness(), "name": "betweenness"}]

# Eigenvector centrality
centralities += [
    {"centrality": g.eigenvector_centrality(), "name": "eigenvector centrality"}
]

In [ ]:
from scipy import stats

deg = g.degree()

correlation = {c["name"]: stats.pearsonr(deg, c["centrality"])[0] for c in centralities}
correlation

All centrality metrics except few metrics (e.g., closeness) are strongly correlated with degree.

## Debiasing degree bias

If your network has a heterogeneous degree distribution, you should consider whether a focal structure is explained by the degree to what extent. Many structures can be explained by degree, such as [nested structure](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.9.031024) and [core-periphery structure](https://iopscience.iop.org/article/10.1088/1367-2630/aab547#:~:text=In%20other%20words%2C%20a%20core,overlap%20with%20the%20first%20one.).

Like the normalized rich club coefficient, the discounting degree effect can bring out new insights. For example, the modularity maximization naturally discounts the degree effect since it seeks the communities that are denser than expected by the configuration model. Similarly, [the degree-corrected stochastic block model](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.83.016107) discounts the effect of the degree, finding a more natural community structure.

#### An origin of degree heterogeneity

The rich club phenomenon can be explained largely by the heterogeneous degree distribution. And the heterogeneous degree distribution is, in fact, ubiquitous across a variety of networks. But what is, in the first place, the underlying mechanism generating this heterogeneity?

We can approach the "cause" by thinking about yet another network process, i.e., preferential attachment.
It is a process in which a node is more likely to form a new edge with another node with a high degree. It is also called the `rich-gets-richer` phenomenon since a high-degree node is more likely to be connected than less cited nodes.

In [ ]:
def preferential_attachment_model(n, m, n0=20, m0=1):
    """
    Generate a graph using the preferential attachment model. The network is unweighted and undirected.

    Parameters
    ----------
    n : int
        The number of new nodes at each time step.
    m : int
        The number of edges to attach from a new node to existing nodes.
    n0 : int
        The initial number of nodes in the network.
    m0 : int
        The offset number of edges.

    Returns
    -------
    A : scipy.sparse matrix
        Adjacency matrix of the generated network.

    Hint:
    - At first, create a star graph of n0 nodes, where one node connects to every other node, but every other node is not connected.
    - Then, for iterate over n - n0 time steps. At each time step, add a new node and connect it to m existing nodes.
    - The probability of connecting a new node to an existing node is proportional to the degree plus m0 of the existing node.
    - To sample a node based on the probability, you can use `np.random.choice`. Use `replace=False` to avoid multiple edges between the same pair of nodes.
    """
    ...

How heterogeneous is the degree distribution? Let's plot the CCDF!

In [ ]:
# Generate random network
n = A.shape[0]

Arand = preferential_attachment_model(n=n, m=5, n0=10, m0=1)

# Compute the degree
deg_rand = np.array(Arand.sum(axis=0)).ravel()

# Plot the degree distributions
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax = sns.ecdfplot(
    deg_rand,
    label="Preferential attachment model",
    complementary=True,
    log_scale=(True, True),
    ax=ax,
)
ax.legend(frameon=False)
ax.set_xlabel("Degree")
sns.despine()

# Assignment
**Q1 Draw the rich club coefficient as a function of $k$ for an Erdős–Rényi random graph of 5000 nodes with the edge probability of $p=1/500$. You can use igraph.API, e.g., `igraph.Graph.Erdos_Renyi(n=15, p=0.2, directed=False, loops=False)`**

---

**Q2: Plot the rich club coefficient against the degree ($k$) for both the original Internet network and the configuration model that precisely preserves the degree. The configuration model can be generated using the 'g.rewire' function. Please note that the 'g.rewire' API is an in-place operation, so create a copy of the network before executing the .rewire API.**

---

**Q3: Compute the normalized rich club coefficient numerically by using 30 realizations of the configuration model. And draw a figure of the degree vs the degree-normalized rich club coefficient**

---

**Q4: Implement the preferential attachment model below. And generate a network based on the preferential attachment model with $m=5$ and $n$ being the number of nodes in the Internet network. Then, plot the CCDF of the degree distributions for the Internet network and the preferential attachment in the same plot in a log-log scale. Do not forget to label your x and y axes and clearly indicate which distribution corresponds to which model.**

In [ ]:
def preferential_attachment_model(n, m, n0=20, m0=1):
    """
    Generate a graph using the preferential attachment model. The network is unweighted and undirected.

    Parameters
    ----------
    n : int
        The number of new nodes at each time step.
    m : int
        The number of edges to attach from a new node to existing nodes.
    n0 : int
        The initial number of nodes in the network.
    m0 : int
        The offset number of edges.

    Returns
    -------
    A : scipy.sparse matrix
        Adjacency matrix of the generated network.

    Hint:
    - At first, create a star graph of n0 nodes, where one node connects to every other node, but every other node is not connected.
    - Then, for iterate over n - n0 time steps. At each time step, add a new node and connect it to m existing nodes.
    - The probability of connecting a new node to an existing node is proportional to the degree plus m0 of the existing node.
    - To sample a node based on the probability, you can use `np.random.choice`. Use `replace=False` to avoid multiple edges between the same pair of nodes.
    """
    ...

You should see a heterogeneous degree distribution with a fat-trail distribution similar to the one for the citation network. Although the distribution is different, it's remarkable that a simple model like the preferential attachment model can reproduce the heterogeneous degree distribution to a large extent.